# Introduction to physrisk

## A brief overview
Physrisk comprises:

* A `HazardModel` that retrieves *hazard indicators* for different locations.
* `VulnerabilityModel` instances that assess the vulnerability of assets to different climate hazards. `VulnerabilityModel` instances use hazard indicators requested from the `HazardModel` to calculate the *impact* of a hazard on a collection of assets.
* Financial models that use the impacts calculated by the `VulnerabilityModels` to calculate risk measures and scores.

`VulnerabilityModels` request hazard indicators using an `indicator_id` (e.g. 'flood_depth' for inundation, 'max_speed' for wind). It is the responsibility of the `HazardModel` to select the source of the hazard indicator data. Although `VulnerabilityModels` can request specific hazard indicators by using a `HazardDataHint` (e.g. a flood depth data set generated by a particular organization), this is generally discouraged: the idea is that a vulnerability model works with any data set. Note that units of the quantity are provided to the `VulnerabilityModel` by the `HazardModel`.

## Hazard indicator data sets
The `HazardModel` retrieves hazard indicators in a number of ways and can be made composite in order to combine different ways of accessing the data. At time of writing the common cases are that:

1. Hazard indicator data is stored in [Zarr](https://zarr.readthedocs.io/en/stable/) format (in an arbitrary Zarr store, although S3 is a popular choice).
2. Hazard indicator data is retrieved via call to an external API. This is mainly used when combining commercial data to the public-domain.

In case 1, hazard indicators are stored as three dimensional arrays. The array is ordered $(z, y, x)$ where $y$ is the spatial $y$ coordinate (e.g. latitude), $x$ is the spatial $x$ coordinate (e.g. longitude) and $z$ is an *index* coordinate. The *index* takes on different meanings according to the type of data being stored.

Indicators can be either:

* Acute (A): the data comprises a set of hazard intensities for different return periods. In this case *index* refers to the different return periods.
* Parametric (P): the data comprises a set of parameters. Here *index* refers to the different parameters. The parameters may be single values, or *index* might refer to a set of thresholds. Parametric indicators are used for chronic hazards.

As mentioned above, a `VulnerabilityModel` only specifies the identifier of the hazard indicator that is required, as well as the climate scenario identifier and the year of the future projection (in the common case where hints are not used). This means that hazard indicator identifier uniquely defines the data. For example, a vulnerability model requesting 'flood depth' could have data returned from a variety of data sets, depending on how the `HazardModel` is configured.

In case of stored-down (or 'pre-generated) hazard indicator data, as opposed to data source via an API, configuring which data sets to use is done by defining a `SourcePath`. This is a function that provides the path, a unique identifier, that provides the data set to be used based on the hazard indicator ID, scenario and year. A convenience class `CoreInventorySourcePaths` creates `SourcePath` instances based on a set of rules called 'selectors'. A selector is a rule such as 'use the flood data set from this particular institution'. 

Some examples of hazard indicators are given in the following table. The naming convention is that underscore '/' should be used in identifiers to separate out distinct and interchangeable elements of the definition (e.g. whether a count is number of days *above* or *below* a threshold). Underscore is used when a single element has multiple words, e.g. 'fire probability' or 'flood depth'. 

| Hazard class | Indicator ID (type) | Description |
|---|---|---|
| CoastalInundation, <br>PluvialInundation, <br>RiverineInundation | flood_depth (A)      | Unprotected flood depth (m) for available return periods. |
| CoastalInundation, <br>PluvialInundation, <br>RiverineInundation | sop (P)              | Standard of protection as return period in years. |
| Fire | fire_probability (P) | Annual probability that location is in a wildfire zone. |
| ChronicHeat | mean_degree_days/above/index (P) | Mean mean-temperature degree days per year above a set of thresholds (specified by the 'index'). |
| ChronicHeat | days/tas/above/index (P) | Mean days per year with daily mean near-surface air temperature above a set of thresholds. |
| ChronicHeat | days/wbgt/above/index (aka days_wbgt_above) (P) | Mean days per year with daily mean wet bulb globe temperature above a set of thresholds. |
| Drought | months/spei/12m/below/index (P) | Mean months per year where the 12 month SPEI index is below a set of thesholds. |   
| Hail | days/above/5cm (P) | Expected days per year for which hail with diameter greater than 5 cm can occur. |                               
| Wind | max_speed (A) | Maximum 1 minute sustained wind speed for available return periods at 10 m height. |
| Drought | shrink_swell/susceptability_score (P) | Score (1-5), typically based on soils’ clay content and soil moisture changes. |
| Subsidence | land_subsidence_rate (P) | Land subsidence rate (millimetres/year). |
| Landslide | susceptability_score (P) | Score (1-5), typically based on characteristics of the terrain combined with daily maximum precipitation (per return period). |
| WaterStress | water_stress (P) | Ratio of water demand to water supply. |

## Event based modelling
The most common use case for physrisk at time of writing is to perform analyses of portfolios but with asset impacts treated separately or with heuristics defining the dependence of asset impacts. In such cases it is possible, and indeed efficient, to derive the marginal probability distribution of the impact of the asset, the `ImpactDistrib`. However to capture more realistic dependence of impacts, event-based modelling is needed. Here the `HazardModel` additionally supplies an array of simulated hazard indicator values to the `VulnerabilityModel` for each asset location, which in turn samples an array of impacts from the vulnerability function. At time of writing, the event-based functionality is not merged into main. Although specific types of `HazardModel` and `VulnerabilityModel` are needed for the event-based case, these do not replace the existing calculation which is appropriate for the separate-asset case: event-based analyses are typically much more computationally intensive. 

## Vulnerability models and configuration
Although it is possible to create bespoke vulnerability models programmatically within physrisk (i.e. adding a new vulnerability model class in code), vulnerability models for different hazards and types of asset are similar enough that it is possible to create generic vulnerability models that are customized via configuration (see Vulnerability Configuration section).